In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from networkx.algorithms import bipartite
import numpy as np

In [ ]:
user_df = pd.read_json('user.json', lines=True)
user_df.head()

In [ ]:
user_df[['elite', '', '', '', '', '', '', '']]

In [2]:
business_df = pd.read_json('business.json', lines=True)
business_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


In [4]:
tip_df = pd.read_json('tip.json', lines=True)
tip_df.head()

,business_id,compliment_count,date,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ


In [3]:
business_df['categories'].value_counts().head(60)

Restaurants, Pizza                                    1042
Nail Salons, Beauty & Spas                            1031
Pizza, Restaurants                                     993
Beauty & Spas, Nail Salons                             947
Food, Coffee & Tea                                     888
Mexican, Restaurants                                   885
Coffee & Tea, Food                                     865
Restaurants, Mexican                                   853
Chinese, Restaurants                                   840
Hair Salons, Beauty & Spas                             831
Beauty & Spas, Hair Salons                             819
Restaurants, Chinese                                   789
Automotive, Auto Repair                                585
Auto Repair, Automotive                                534
Food, Grocery                                          492
Grocery, Food                                          491
Restaurants, Italian                                   4

In [4]:
business_df[['address', 'categories', 'city', 'name', 'postal_code', 'review_count', 'stars', 'state']]

,address,categories,city,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,"Golf, Active Life",Phoenix,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","Sushi Bars, Restaurants, Japanese",Charlotte,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237","Insurance, Financial Services",Goodyear,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","Plumbing, Shopping, Local Services, Home Servi...",Charlotte,Queen City Plumbing,28217,4,4.0,NC
5,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd","Shipping Centers, Couriers & Delivery Services...",Mississauga,The UPS Store,L5V 0B1,3,2.5,ON
6,20 Douglas Woods Drive Southeast,"Beauty & Spas, Hair Salons",Calgary,Edgeworxx Studio,T2Z 1K4,7,3.5,AB
7,"4545 E Tropicana Rd Ste 8, Tropicana","Hair Salons, Hair Stylists, Barbers, Men's Hai...",Las Vegas,Supercuts,89121,3,3.5,NV
8,5940 W Union Hills Dr,"Nail Salons, Beauty & Spas, Day Spas",Glendale,Vita Bella Fine Day Spa,85308,8,5.0,AZ
9,21689 Lorain Rd,"Beauty & Spas, Nail Salons, Day Spas, Massage",Fairview Park,Options Salon & Spa,44126,8,4.5,OH


In [5]:
cityisMontreal = business_df['city']=='MontrÃ©al'
print(cityisMontreal.head())

0    False
1    False
2    False
3    False
4    False
Name: city, dtype: bool


In [7]:
MontrealBusiness = business_df[cityisMontreal]
MontrealBusiness.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
82,8261 Saint-Laurent Boulevard,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",i_oghUPqLzzJtxC6Zm-D2A,"Canadian (New), Portuguese, Restaurants",MontrÃ©al,"{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...",1,45.539733,-73.633639,Braseiro,H2P 2M1,22,4.0,QC
84,2425 Boulevard Gouin E,"{'GoodForKids': 'True', 'BikeParking': 'True',...",735UkyT4px_oeW3iyotGmA,"Parks, Active Life",MontrÃ©al,"{'Monday': '9:0-16:30', 'Tuesday': '9:0-16:30'...",1,45.583898,-73.653778,Parc-nature de l'ÃŽle-de-la-Visitation,H2B 1X7,10,4.5,QC
92,10897 Rue du Pont,"{'OutdoorSeating': 'True', 'WiFi': 'u'free'', ...",SWX0K1dpy4fcRq-ktZxmCg,"Landmarks & Historical Buildings, Mediterranea...",MontrÃ©al,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.574857,-73.661130,Bistro des Moulins,H2B 2H3,3,3.5,QC
180,134 Rue Jean-Talon O,"{'OutdoorSeating': 'False', 'RestaurantsPriceR...",spi9UEWyNrq2O4y5DD84LA,"Vietnamese, Restaurants, Soup",MontrÃ©al,"{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",0,45.533111,-73.619518,Restaurant Nhu Y,H2R 2X1,11,4.0,QC
194,5411 rue Beaubien E,{'ByAppointmentOnly': 'True'},cBQDYO0dIpG6--I3DMiWoQ,"Hair Removal, Beauty & Spas, Shopping, Laser H...",MontrÃ©al,"{'Tuesday': '10:0-18:0', 'Wednesday': '10:0-18...",1,45.575935,-73.568558,Secrets de Femme,H1T 1W7,3,1.0,QC


In [8]:
MontrealBusiness.shape

(6449, 14)

In [9]:
MontrealBusiness.city = MontrealBusiness.city.replace({'MontrÃ©al':'Montreal'})
MontrealBusiness.tail(100)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
189883,600 Chabanel W,{'RestaurantsPriceRange2': '3'},vbWQvKr3fl-RvogVeLXk3g,"Shopping, Fashion",Montreal,None,1,45.537531,-73.655116,Soia & Kyo,H2N 2K6,4,2.0,QC
189943,3575 Du Parc Av,None,uIlcXE_nmSYWRxImDGP1Lg,"American (New), Canadian (New), Restaurants",Montreal,None,0,45.510344,-73.575114,Boccacino's Restaurant,H2X 3P9,3,3.5,QC
189988,288 Rue Laurier O,"{'Alcohol': 'u'full_bar'', 'Ambience': '{'roma...",GT4Tn-tNZbKaoQhfKtHYZg,"Tapas Bars, Restaurants, Sushi Bars, Asian Fusion",Montreal,"{'Tuesday': '17:30-23:0', 'Wednesday': '17:30-...",0,45.519882,-73.595542,Ryu,H2V 2K2,10,3.5,QC
190002,5731 Park Avenue,{'HasTV': 'False'},2ucxVuwDJDFfokHvgnjsMg,"Vegetarian, Desserts, Cafes, Restaurants, Food",Montreal,"{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...",0,45.523567,-73.605229,CafÃ© Zosha,H2V 4G9,4,4.0,QC
190031,355 Rue Saint-Jacques,"{'RestaurantsPriceRange2': '4', 'WiFi': ''free''}",BcsGNJOqIxF8U6kL_zCyGQ,"Hotels, Hotels & Travel, Event Planning & Serv...",Montreal,None,1,45.502682,-73.559485,HÃ´tel le St-James,H2Y 1N9,21,4.5,QC
190042,1246 Rue Sainte-Catherine O,"{'BusinessParking': '{'garage': False, 'street...",0Zj7GiJW4LVs7VaeDZuxzA,"Men's Clothing, Shopping, Vinyl Records, Books...",Montreal,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,45.498017,-73.574642,Urban Outfitters,H3G 1P1,14,3.0,QC
190058,"1619 Rue William, Suite B","{'RestaurantsAttire': 'u'casual'', 'Restaurant...",7bdU3QMrMHwvm9D2H6VV9A,"Wine Bars, Restaurants, Turkish, Bars, Breakfa...",Montreal,"{'Thursday': '18:0-2:0', 'Friday': '18:0-2:0',...",1,45.488350,-73.566037,Arem Restaurant,H3J 1R1,3,3.0,QC
190060,393 Rue Saint-Paul Est,"{'RestaurantsGoodForGroups': 'True', 'Restaura...",6XZATxc2fcBoSjit1pVDwA,"Salad, Bakeries, Sandwiches, Restaurants, Heal...",Montreal,"{'Monday': '11:0-18:0', 'Tuesday': '11:0-18:0'...",1,45.509835,-73.551541,7Grains Bakery & CafÃ©,H2Y 1H3,24,2.0,QC
190067,5673 Rue Sherbrooke Ouest,"{'GoodForKids': 'True', 'BikeParking': 'True',...",A_Gl8o6xlhugD0r8F99QoQ,"Coffee & Tea, Food, Cafes, Restaurants",Montreal,"{'Monday': '9:0-19:0', 'Tuesday': '9:0-19:0', ...",1,45.470956,-73.615396,Kokkino Cafe,H4A 2X7,7,3.5,QC
190135,3556 Boulevard St-Laurent,"{'GoodForDancing': 'True', 'RestaurantsPriceRa...",2POlhGAcAvUOLIzZnv_Unw,"Bars, Nightlife",Montreal,None,1,45.513314,-73.571850,Nightclub Ivy,H2X 2T6,4,1.0,QC


In [40]:
def Coffee(MontrealBusiness):
    if 'Parks, Active Life' in 'categories':
        return 'Coffee & Tea'

In [41]:
print(Coffee(MontrealBusiness))

None


In [10]:
MontrealBusiness=MontrealBusiness[["attributes", 'categories', 'city', 'name', 'review_count','stars']]

In [44]:
MontrealBusiness.head()

,attributes,categories,city,name,review_count,stars
82,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...","Canadian (New), Portuguese, Restaurants",Montreal,Braseiro,22,4.0
84,"{'GoodForKids': 'True', 'BikeParking': 'True',...","Parks, Active Life",Montreal,Parc-nature de l'ÃŽle-de-la-Visitation,10,4.5
92,"{'OutdoorSeating': 'True', 'WiFi': 'u'free'', ...","Landmarks & Historical Buildings, Mediterranea...",Montreal,Bistro des Moulins,3,3.5
180,"{'OutdoorSeating': 'False', 'RestaurantsPriceR...","Vietnamese, Restaurants, Soup",Montreal,Restaurant Nhu Y,11,4.0
194,{'ByAppointmentOnly': 'True'},"Hair Removal, Beauty & Spas, Shopping, Laser H...",Montreal,Secrets de Femme,3,1.0


In [11]:
MontrealBusiness['categories'].value_counts().head(80)

French, Restaurants                                   61
Food, Coffee & Tea                                    59
Pizza, Restaurants                                    57
Restaurants, Italian                                  55
Restaurants, French                                   52
Chinese, Restaurants                                  46
Restaurants, Pizza                                    44
Restaurants, Chinese                                  43
Coffee & Tea, Food                                    40
Italian, Restaurants                                  38
Food, Bakeries                                        38
Sushi Bars, Restaurants                               37
Vietnamese, Restaurants                               36
Bakeries, Food                                        34
Restaurants, Sushi Bars                               34
Cafes, Restaurants                                    30
Restaurants, Cafes                                    28
Restaurants, Breakfast & Brunch

In [ ]:
templist

In [18]:
MontrealBusiness_Coffee = MontrealBusiness[MontrealBusiness['categories'].isin(['Coffee & Tea, Food', 'Food, Coffee & Tea'])]
MontrealBusiness_Coffee.head(60)
#MontrealBusiness_Coffee.shape


,attributes,categories,city,name,review_count,stars
4535,"{'WheelchairAccessible': 'True', 'RestaurantsT...","Food, Coffee & Tea",Montreal,CafÃ© Reine GarÃ§on,9,4.0
5389,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...","Food, Coffee & Tea",Montreal,Presse CafÃ©,16,3.5
7576,"{'BusinessParking': '{'garage': False, 'street...","Food, Coffee & Tea",Montreal,Starbucks,10,4.5
7928,"{'BusinessParking': '{'garage': False, 'street...","Food, Coffee & Tea",Montreal,Cafe Momus,3,4.5
13787,"{'WiFi': ''free'', 'RestaurantsPriceRange2': '...","Food, Coffee & Tea",Montreal,Second Cup,12,4.0
15266,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Food, Coffee & Tea",Montreal,Lapin PressÃ©,33,4.0
18014,"{'WiFi': ''free'', 'RestaurantsPriceRange2': '...","Food, Coffee & Tea",Montreal,Tim Hortons,3,3.0
19153,"{'BikeParking': 'False', 'WiFi': ''free'', 'Bu...","Food, Coffee & Tea",Montreal,Starbucks,14,2.5
19595,"{'RestaurantsTakeOut': 'True', 'WiFi': 'u'free...","Food, Coffee & Tea",Montreal,Starbucks,6,3.5
24164,"{'BusinessParking': '{'garage': False, 'street...","Coffee & Tea, Food",Montreal,Dispatch Coffee,26,4.5


In [13]:
MontrealBusiness.groupby('name').stars.mean()

name
'Round Table Tours                            5.00
'Turel                                        4.00
1000 Grammes                                  3.50
1000000 Comix                                 3.00
123 Bulgogi                                   3.00
155 Fahrenheit                                4.50
180g                                          5.00
1909 Taverne Moderne                          3.00
2 Cochons                                     3.50
2 Pierrots                                    4.00
2K Fusion                                     2.50
3 Amigos                                      2.50
3 Amigos Cantina Grill                        3.00
3 Minots                                      2.00
3734                                          4.50
3D Eyebrow Studio                             4.00
3G Mobiletech                                 1.00
4 Origines                                    4.50
4 U Tea House                                 4.00
4saisons Sushi            